Importing libaries and defining relvent constants, we work in natrual units so hbar=c=1

In [108]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

HBAR = 1 # Left in natrul units, 6.582 * 10**(-22) in MeV s
REDUCED_MASS = 10/11 * 931.5  # In Mev/c^2
ORBITAL_QUANTUM_NUMBER = 0  # In units of hbar?
SUM_LIMIT = 5
POTENTIAL_PARAMETERS = [40, 0.2]  # In [Mev, fm^-2]
INITIAL_RANGE_PARAMETER = 0.01  # In fm^-2
radius, E = sympy.symbols('radius E')

We define several key functions here. These are the wavefunction in terms of a gaussian and range parameter, which allows us to take it as a series expansion in terms of N gausians, the gaussian potential we use as a first (analytic) guess, and the kinetic energy operator which appears in the hamiltonian.

In [109]:
def gaussian_wave_function(radius, range_parameter, orbital_quantum_number=ORBITAL_QUANTUM_NUMBER):
    """
    Defines the form of the gaussian wavefunctions used in the series expansion of the wavefunction in terms of the radius r,
    the oribtal angular momentum, l, and a range parmeter, α_i.

    Parameters
    ----------
    radius : float
    range_parameter : float characterizing the gaussian

    Returns
    -------
    float the value of the gaussian wavefunction at the radius r
    """
    return radius**(orbital_quantum_number) * sympy.exp(- range_parameter * radius**2)


def gaussian_potential(radius, parameters=POTENTIAL_PARAMETERS):
    """
    Defines the spherically symmetric gaussian potential as a function of radius we use as a 'first guess'
    to make our integrals analytic. Given as V = V_0 * e**(-beta * radius**2)

    Parameters
    ----------
    radius : float
    paramaters : 1d array containg the fixed paramaters for our potential. In the order [V_0, beta] (see slides/ suplimentry notes) 

    Returns
    -------
    float with the value of the potential at a radius r
    """
    return parameters[0] * sympy.exp(-1 * parameters[1] * radius**2)


def kinetic_energy_operator(radius, wavefunction, range_parameter, hbar=HBAR, reduced_mass=REDUCED_MASS,
                            orbital_quantum_number=ORBITAL_QUANTUM_NUMBER):
    return - hbar**2 / (2 * reduced_mass) * sympy.diff(wavefunction(radius, range_parameter), radius, radius) + (
        orbital_quantum_number * (orbital_quantum_number + 1) * hbar**2) / (2 * reduced_mass * radius**2)


def next_range_parameter(i, initial_range_parameter=INITIAL_RANGE_PARAMETER, some_number=2):
    new_range_parameter = initial_range_parameter * some_number**(i-1)
    return new_range_parameter

We now define our matrix elements

In [110]:
def N_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2):
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * wavefunction_2(radius, range_parameter_2), (radius, 0, sympy.oo))


def T_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2):
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * kinetic_energy_operator(radius, wavefunction_2, range_parameter_2),
                           (radius, 0, sympy.oo))


def V_ij(radius, wavefunction_1, wavefunction_2, range_parameter_1, range_parameter_2, potential_function):
    return sympy.integrate(wavefunction_1(radius, range_parameter_1) * potential_function(radius) * wavefunction_2(radius, range_parameter_2),
                           (radius, 0, sympy.oo))


We now need to create the H_ij and N_ij matricies and solve the characteristic equation for a GENERALISED eigenvalue problem, and save this as a variable.


In [114]:
h_matrix = sympy.zeros(SUM_LIMIT - 1, SUM_LIMIT - 1)
n_matrix = sympy.zeros(SUM_LIMIT - 1, SUM_LIMIT - 1)
i = 1
j = 1
i_range_parameter = INITIAL_RANGE_PARAMETER
j_range_parameter = INITIAL_RANGE_PARAMETER

while i < SUM_LIMIT:
    while j < SUM_LIMIT:
        h_matrix[i-1, j-1] = T_ij(radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter) + V_ij(
            radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter, gaussian_potential)
        n_matrix[i-1, j-1] =N_ij(radius, gaussian_wave_function, gaussian_wave_function, i_range_parameter, j_range_parameter)
        j += 1
        j_range_parameter = next_range_parameter(j)
        print("row", i, "column", j-1, "complete")
    i += 1
    i_range_parameter = next_range_parameter(i)
    j = 1
    j_range_parameter = INITIAL_RANGE_PARAMETER
print("Both matricies found")
    
# returns system of expressions, set equal to zero
# use sympy.solve([insert equations here, comma separated], e) to solve for energy


row 1 column 1 complete
row 1 column 2 complete
row 1 column 3 complete
row 1 column 4 complete
row 2 column 1 complete
row 2 column 2 complete
row 2 column 3 complete
row 2 column 4 complete
row 3 column 1 complete
row 3 column 2 complete
row 3 column 3 complete
row 3 column 4 complete
row 4 column 1 complete
row 4 column 2 complete
row 4 column 3 complete
row 4 column 4 complete
Both matricies found


We now seek to solve the generalised bad boy (?)


In [115]:
generalised_eigenvalues = sympy.solve(sympy.det(h_matrix - E * n_matrix), E)
print(generalised_eigenvalues)


[0.000354787171881519, 0.101983159431602, 4.59001411726263, 31.4574568592418]


We now want the generalised eigenvectors (for which we havent found a simple way to do)

In [ ]:
for element in enumerate(generalised_eigenvalues)